In [ ]:
import pandas as pd
import numpy as np

# Firt Assignment

### a) Look at the dataset

In [3]:
#read the table of ratings(so we get the structure and the dimentions)
ratings=pd.read_csv('ml-latest-small/ratings.csv')
print('number of ratings:', len(ratings))
ratings

number of ratings: 100836


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931
...,...,...,...,...
100831,610,166534,4.0,1493848402
100832,610,168248,5.0,1493850091
100833,610,168250,5.0,1494273047
100834,610,168252,5.0,1493846352


In [4]:
movies=pd.read_csv('ml-latest-small/movies.csv')
movies

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy
...,...,...,...
9737,193581,Black Butler: Book of the Atlantic (2017),Action|Animation|Comedy|Fantasy
9738,193583,No Game No Life: Zero (2017),Animation|Comedy|Fantasy
9739,193585,Flint (2017),Drama
9740,193587,Bungo Stray Dogs: Dead Apple (2018),Action|Animation


### b) User-based collaborative filtering approach

b.1)Generate the matrix 

In [5]:

# Determin matrix user-movie (pivot table)
movie_matrix = ratings.pivot_table(index='movieId', columns='userId', values='rating')

movie_matrix


userId,1,2,3,4,5,6,7,8,9,10,...,601,602,603,604,605,606,607,608,609,610
movieId,,,,,,,,,,,,,,,,,,,,,
1,4.0,NaN,NaN,NaN,4.0,NaN,4.5,NaN,NaN,NaN,...,4.0,NaN,4.0,3.0,4.0,2.5,4.0,2.5,3.0,5.0
2,NaN,NaN,NaN,NaN,NaN,4.0,NaN,4.0,NaN,NaN,...,NaN,4.0,NaN,5.0,3.5,NaN,NaN,2.0,NaN,NaN
3,4.0,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,5.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,3.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
193581,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193583,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
193585,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


b.2) implementation of Pearson correlation

In [6]:
#The Pearson correlation coefficient measures the linear relationship between two datasets. 
def pearson_similarity(ratings1, ratings2):
    # movies rated from both users
    movies = ratings1.index.intersection(ratings2.index)
    if len(movies) == 0:
        return 0  # No correlation
    
    user1_ratings = ratings1[movies]
    user2_ratings = ratings2[movies]

    # determine the avarage
    mean_user1 = user1_ratings.mean()
    mean_user2 = user2_ratings.mean()

    # summerize the product of the pair of the normilazed ratings
    numerator = ((user1_ratings - mean_user1) * (user2_ratings - mean_user2)).sum()
    #the squared deviation scores of your variables
    denominator = ((((user1_ratings - mean_user1) ** 2).sum())**0.5) * ((((user2_ratings - mean_user2) ** 2).sum())**0.5)
    if denominator == 0:
        return 0
    else:
        return numerator / denominator

### e) New function for the similarity

In [7]:
#Compute the positional intersection of two sets.Returns a set containing elements that are equal and in the same position in both sets.
def positional_intersection(list1, list2):
    intersection = []
    min_length = min(len(list1), len(list2))
    for i in range(min_length):
        if list1[i] == list2[i]:
            intersection.append(list1[i])
    return intersection

#return the number of element un the union between lists
def union_fun (list1, list2):
    min_length = min(len(list1), len(list2))
    union = len(list1) + len(list2)
    
    for i in range(min_length):
        if list1[i] == list2[i]: #if the is a two times the same ratings, one is not counted
            union-=1
    return union

#jaccard similarity (I choose this similarity because is the most famous that I know of )
#but if dataset is sparse, meaning many users have rated only a few items, it's more likely to observe high similarity scores 
#because there are fewer items to differentiate users. So a lot of users have the same Jaccard similarity of 1 if those items are rated identically.
def jaccard_similarity(target_user_ratings, other_user_ratings):
    # Remove the nan
    target_ratings = target_user_ratings.dropna().tolist()
    other_ratings = other_user_ratings.dropna().tolist()

    intersection =len(positional_intersection(target_ratings, other_ratings))
    union = union_fun(target_ratings, other_ratings)
    if not union:  # Check if the union is empty to avoid division by zero
        return 0
    # Calculate Jaccard similarity
    similarity = intersection / union
    
    return similarity

In [8]:
# Function to compute similarity with a specific user
def similarity_with_user(userId, movie_matrix):
    # Extract ratings of the target user from the movie ratings DataFrame
    target_user_ratings = movie_matrix[userId]
    
    # Compute similarity between the target user and all other users 
    user_similarity_with_other = {user: pearson_similarity(target_user_ratings, movie_matrix[user]) for user in movie_matrix.columns if user != userId}
    # Decomment the following code and comment the above one, if you want the jaccard similarity 
    #user_similarity_with_other = {user: jaccard_similarity(target_user_ratings, movie_matrix[user]) for user in movie_matrix.columns if user != userId}

    # Return the dictionary containing similarities with other users
    return user_similarity_with_other


### c) Function Prediction

In [9]:
# Prediction function
def predict_movie_rating(userId, movieId, ratings, user_similarity_with_other):
    # Filter ratings for the target user
    user_ratings = ratings[ratings['userId'] == userId].set_index('movieId')['rating']
    # Filter ratings for the movie in question
    movie_ratings = ratings[ratings['movieId'] == movieId].set_index('userId')
    # Select only the top 10 most similar users who have rated the movie
    similar_users = [(user, user_similarity_with_other[user]) for user in movie_ratings.index]
    similar_users = sorted(similar_users, key=lambda x: x[1], reverse=True)[:10]
    numerator = 0
    denominator = 0
    
    # Iterate over the top similar users
    for user, similarity in similar_users:
        # Check if similarity is positive
        if similarity > 0:
            # Retrieve the rating and mean rating of the current user
            rating = movie_ratings.loc[user, 'rating']
            mean_rating = movie_ratings['rating'].mean()
            # Update numerator and denominator
            numerator += similarity * (rating - mean_rating)
            denominator += abs(similarity)
                
    # Check if denominator is zero
    if denominator == 0:
        return np.nan  # No similar users found
    else:
        # Compute the predicted rating
        user_mean_rating = user_ratings.mean()
        predicted_rating = user_mean_rating + (numerator / denominator)
        return predicted_rating

    


### d) Texting for the top 10 users and movies

In [10]:
# Example usage:
userId = 3
movieId = 2
user_similarity_with_other = similarity_with_user(userId, movie_matrix)
predicted_rating = predict_movie_rating(userId, movieId, ratings, user_similarity_with_other)
print("Predicted rating for user", userId, "on movie", movieId, ":", predicted_rating)
sorted_users = sorted(user_similarity_with_other.items(), key=lambda x: x[1], reverse=True)
top_similar_users = sorted_users[:10]
print(top_similar_users)

Predicted rating for user 3 on movie 2 : 2.674480351132912
[(441, 0.11741841994010987), (496, 0.06787752167982902), (549, 0.06400609969213772), (231, 0.061158903864791336), (527, 0.058455503222874745), (537, 0.058071938550124255), (313, 0.055313283670736894), (518, 0.05028788491396018), (244, 0.049511365158064743), (246, 0.04831391639756525)]


In [11]:
from joblib import Parallel, delayed

def predict_movie_rating_wrapper(userId, movieId, ratings, user_similarity_with_other):
    return predict_movie_rating(userId, movieId, ratings, user_similarity_with_other)

def predicting_best_movies(userId, ratings, user_similarity_with_other):
    # Get movies not rated by target user
    user_column = movie_matrix[userId]
    movies_to_recommend = user_column[user_column.isna()].index.tolist()
    
    # Create a DataFrame with movies to recommend
    df_to_recommend = pd.DataFrame({'movieId': list(movies_to_recommend)})
    
    # Calculate predicted ratings for movies to recommend (bottleneck)
    # Calculate predicted ratings for movies to recommend in parallel
    predicted_ratings = Parallel(n_jobs=-1)(
        delayed(predict_movie_rating_wrapper)(userId, movieId, ratings, user_similarity_with_other)
        for movieId in df_to_recommend['movieId']
    )
    df_to_recommend['predicted_rating'] = predicted_ratings
    
    # Filter out NaN predicted ratings
    df_to_recommend.dropna(inplace=True)
    
    # Sort recommended movies in descending order of predicted rating
    df_to_recommend.sort_values(by='predicted_rating', ascending=False, inplace=True)
    
    return df_to_recommend[['movieId', 'predicted_rating']].values.tolist()

In [12]:
# Find the top similar users for the target user
userId = 6
user_similarity_with_other = similarity_with_user(userId, movie_matrix)
sorted_users = sorted(user_similarity_with_other.items(), key=lambda x: x[1], reverse=True)
top_similar_users = sorted_users[:10]

print("Top 10 similar users for user", userId, ":")
for user_id, similarity_score in top_similar_users:
    print("User:", user_id, "--> Similarity Score:", similarity_score)

movies_suggested= predicting_best_movies(userId, ratings, user_similarity_with_other)
print("\nTop 10 recommended movies for user", userId, ":")
for movie_id, rating in movies_suggested[:10]:
    movie_title = movies[movies['movieId'] == movie_id]['title'].values[0]
    print("Movie:", movie_title, "--> Predicted Rating:", rating)


Top 10 similar users for user 6 :
User: 181 --> Similarity Score: 0.2637487151247747
User: 126 --> Similarity Score: 0.22304421565141244
User: 584 --> Similarity Score: 0.21879858736115554
User: 411 --> Similarity Score: 0.19585559312851336
User: 179 --> Similarity Score: 0.19266917722195656
User: 470 --> Similarity Score: 0.18910950900888213
User: 136 --> Similarity Score: 0.18053250822097758
User: 94 --> Similarity Score: 0.17254150529344278
User: 379 --> Similarity Score: 0.16892695164348245
User: 486 --> Similarity Score: 0.15410600470826327

Top 10 recommended movies for user 6 :
Movie: Exorcist III, The (1990) --> Predicted Rating: 6.493630573248407
Movie: Herbie Goes Bananas (1980) --> Predicted Rating: 5.365969654368543
Movie: Skin I Live In, The (La piel que habito) (2011) --> Predicted Rating: 5.359075166280375
Movie: Young Adult (2011) --> Predicted Rating: 5.243630573248407
Movie: War for the Planet of the Apes (2017) --> Predicted Rating: 5.172357418488705
Movie: Room in R

# Second Assignment

### a) group recommendation

In [13]:
from statistics import mean

#function for average aggregation
def average_aggregation(users):

    #dictionary to store all the movies and the ratings of all the users
    movie_ratings_dict={}
    for user in users:
        #to get the similarity and all the movies recommended
        user_similarity_with_other = similarity_with_user(user, movie_matrix)
        movies_suggested=predicting_best_movies(user, ratings, user_similarity_with_other)
        
        for movieId, rating in movies_suggested:
            if movieId in movie_ratings_dict:
                if isinstance(movie_ratings_dict[movieId], list):
                    movie_ratings_dict[movieId].append(rating)
                else:
                    movie_ratings_dict[movieId] = [movie_ratings_dict[movieId], rating] 
            else:
                movie_ratings_dict[movieId] = rating
    #print(movie_ratings_dict)
                
    #to memorize the average ratings of all movies            
    movie_avg = {}
    for movie, votes in movie_ratings_dict.items():
        if isinstance(votes, list):  # Check if votes is a list
            if len(votes) == len(users): #consider only the movies with prediction from all users
                movie_avg[movie] = mean(votes)
        
        
    #sort it to get the one with the higest avg
    sorted_movie_avg = dict(sorted(movie_avg.items(), key=lambda item: item[1], reverse=True))
    return sorted_movie_avg

users = [1, 3, 480]
average = average_aggregation(users)

# Print only the top 10 movies
top_10_movies = dict(list(average.items())[:10])
print("Top 10 Movies:")
for movie, avg_rating in top_10_movies.items():
    movie_title = movies[movies['movieId'] == movie]['title'].values[0]
    print("Movie:", movie_title, "--> Predicted Rating:", avg_rating)

Top 10 Movies:
Movie: Hush (2016) --> Predicted Rating: 4.971062119440762
Movie: Opera (1987) --> Predicted Rating: 4.91882532435635
Movie: The Cloverfield Paradox (2018) --> Predicted Rating: 4.887814582824376
Movie: Ivan's Childhood (a.k.a. My Name is Ivan) (Ivanovo detstvo) (1962) --> Predicted Rating: 4.882981421284085
Movie: Unbelievable Adventures of Italians in Russia (1974) --> Predicted Rating: 4.609132121155715
Movie: The Great Train Robbery (1903) --> Predicted Rating: 4.56979111350653
Movie: Dragon Ball Z: The History of Trunks (Doragon bôru Z: Zetsubô e no hankô!! Nokosareta chô senshi - Gohan to Torankusu) (1993) --> Predicted Rating: 4.457192915928266
Movie: Skin I Live In, The (La piel que habito) (2011) --> Predicted Rating: 4.457061709642226
Movie: Peeping Tom (1960) --> Predicted Rating: 4.4350452060414165
Movie: To Be or Not to Be (1942) --> Predicted Rating: 4.431458151523233


In [14]:
#use three users
users=[1, 3, 480]

#least misery aggregation consider the user less satisfiaed 
def least_misery_aggregation(users):

    movie_ratings_dict={}
    for user in users:
        user_similarity_with_other = similarity_with_user(user, movie_matrix)
        movies_suggested=predicting_best_movies(user, ratings, user_similarity_with_other)
        
        for movieId, rating in movies_suggested:
            if movieId in movie_ratings_dict:
                if isinstance(movie_ratings_dict[movieId], list):
                    movie_ratings_dict[movieId].append(rating)
                else:
                    movie_ratings_dict[movieId] = [movie_ratings_dict[movieId], rating]
            else:
                movie_ratings_dict[movieId] = rating
    movie_min={}
    for movie, votes in movie_ratings_dict.items():
        if isinstance(votes, list):  # Check if votes is a list
            if len(votes) == len(users): #consider only the movies with prediction from all users
                movie_min[movie] = min(votes)
        
        
    sorted_movie_min = dict(sorted(movie_min.items(), key=lambda item: item[1], reverse=True))
    return sorted_movie_min

least_misery = least_misery_aggregation(users)
# Print only the top 10 movies
top_10_movies = dict(list(least_misery.items())[:10])
print("Top 10 Movies:")
for movie, rat in top_10_movies.items():
    movie_title = movies[movies['movieId'] == movie]['title'].values[0]
    print("Movie:", movie_title, "--> Predicted Rating:", rat)

Top 10 Movies:
Movie: Opera (1987) --> Predicted Rating: 4.435897435897436
Movie: Hush (2016) --> Predicted Rating: 4.269230769230769
Movie: The Cloverfield Paradox (2018) --> Predicted Rating: 4.185897435897436
Movie: Peeping Tom (1960) --> Predicted Rating: 4.002858871881985
Movie: Hachiko: A Dog's Story (a.k.a. Hachi: A Dog's Tale) (2009) --> Predicted Rating: 3.935897435897436
Movie: Skin I Live In, The (La piel que habito) (2011) --> Predicted Rating: 3.9358974358974357
Movie: Witchfinder General (Conquerer Worm, The) (1968) --> Predicted Rating: 3.9358974358974357
Movie: The Great Train Robbery (1903) --> Predicted Rating: 3.9206524589793657
Movie: Witches of Eastwick, The (1987) --> Predicted Rating: 3.7841442921880972
Movie: Maze Runner: Scorch Trials (2015) --> Predicted Rating: 3.7066252076548514


### b) my aggregation

In [15]:
from statistics import mean

#aggregation that use the difference between ratings to order the movies
def weighted_aggregation(users):
    
    movie_ratings_dict={}
    for user in users:
        user_similarity_with_other = similarity_with_user(user, movie_matrix)
        movies_suggested=predicting_best_movies(user, ratings, user_similarity_with_other)
        for movieId, rating in movies_suggested:
            if movieId in movie_ratings_dict:
                if isinstance(movie_ratings_dict[movieId], list):
                    movie_ratings_dict[movieId].append(rating)
                else:
                    movie_ratings_dict[movieId] = [movie_ratings_dict[movieId], rating]
            else:
                movie_ratings_dict[movieId] = rating
    #print(movie_ratings_dict)
    movie_avg={}
    for movie, votes in movie_ratings_dict.items():
        if isinstance(votes, list):  # Check if votes is a list
            if len(votes) == len(users): #consider only the movies with prediction from all users
                avg_rating = mean(votes)
                diff = max(votes) - min(votes) #disagreement between users on a movie rating
                movie_avg[movie] = (avg_rating, diff)
    #oreder on average rating and disagreement between users(disagreement decrease, average rating increase)
    sorted_movie_avg = dict(sorted(movie_avg.items(), key=lambda item: item[1][0] * (1/item[1][1]), reverse=True))
    return sorted_movie_avg

users = [1, 3, 480]
top_mov = weighted_aggregation(users)
 # Print only the top 10 movies
top_10_movies = dict(list(top_mov.items())[:10])
print("Top 10 Movies:")
for movie, (avg_rating, weight) in top_10_movies.items():
    movie_title = movies[movies['movieId'] == movie]['title'].values[0]
    print(f"Movie: {movie_title} --> Avg Rating: {avg_rating}, Weight: {weight}")

Top 10 Movies:
Movie: Joy Ride (2001) --> Avg Rating: 3.456479642400125, Weight: 0.21312798603271643
Movie: I Sell the Dead (2008) --> Avg Rating: 3.5196430889089556, Weight: 0.22748756197518638
Movie: Zombeavers (2014) --> Avg Rating: 3.5196430889089556, Weight: 0.22748756197518638
Movie: Tideland (2005) --> Avg Rating: 3.519858957607847, Weight: 0.23394264430986622
Movie: Hostel: Part II (2007) --> Avg Rating: 3.453501686937617, Weight: 0.23224986355126997
Movie: Shakes the Clown (1992) --> Avg Rating: 3.2068246708506103, Weight: 0.3037833840901434
Movie: Furious 7 (2015) --> Avg Rating: 3.616738974366626, Weight: 0.35935521340790055
Movie: Forget Paris (1995) --> Avg Rating: 3.1647922856653135, Weight: 0.3157907217435163
Movie: Innkeepers, The (2011) --> Avg Rating: 3.489781274565654, Weight: 0.39242381954338645
Movie: Project A 2 ('A' gai wak juk jap) (1987) --> Avg Rating: 3.489781274565654, Weight: 0.39242381954338645


# Third assignment

In [16]:
#compute the satisfation of all users with the movies 
def compute_users_satisfaction(users, top_group):
    
    users_satisfaction=[]
    group_prediction=0 #sum up group prediction for the top movies
    for movie, rat in top_group.items():
        group_prediction += rat

    #for every users determine his sudisfaction
    for user in users:

        users_prediction=0 
        
        #sum up user prediction for the top movies of the group
        user_similarity_with_other = similarity_with_user(user, movie_matrix)
        movies_suggested=predicting_best_movies(user, ratings, user_similarity_with_other)
        for movie, rating in movies_suggested:
            if movie in top_group:
                users_prediction += rating

        users_satisfaction.append(group_prediction/users_prediction)

    return users_satisfaction



def compute_min_max_users_satisfaction(users, top_group):
    
    users_sadisfaction=compute_users_satisfaction(users, top_group)
    return max(users_sadisfaction), min(users_sadisfaction)


#copute the overall satisfaction of the users at a given iteration
def compute_users_overall_satisfaction(satisfactions, iteration):
    users_overall_satisfaction={}
    for user, ratings in satisfactions.items():
        users_overall_satisfaction[user] = sum(ratings) / (iteration+1)
    return users_overall_satisfaction


In [17]:

#the classic version
def seq_aggregation(users, iteration):
    
    top_group={}
    old_iterations_moives={}
    users_satisfaction_iterations = {user: [] for user in range(len(users))}
    #get the two method aggregation
    movies_avg = average_aggregation(users)
    movies_least_misery = least_misery_aggregation(users)
    for iter in range(iteration):
        print("\nIteration: ", iter)
        if iter == 0: #the first iteration use only the average method
            top_group = movies_avg
        else:
            #compute previous iteration satisfction (alfa)
            max_user_satisfaction, min_user_satisfaction = compute_min_max_users_satisfaction(users, top_group)
            alfa = max_user_satisfaction - min_user_satisfaction
            new_recommendation = {}
            for movie, avg_rating in movies_avg.items():
                if movie not in old_iterations_moives: #to consider different movies for the new iteration
                    if movie in movies_least_misery:
                        min_rating = movies_least_misery[movie]
                        new_recommendation[movie] = avg_rating * (1 - alfa) + min_rating * alfa
                        top_group = dict(sorted(new_recommendation.items(), key=lambda item: item[1], reverse=True))

        old_iterations_moives.update(dict(list(top_group.items())[:10])) #to memorize the top movies predicted in the old iterations
        #top movies
        print("Top 10 Movies:")
        for movie, rat in dict(list(top_group.items())[:10]).items():
            movie_title = movies[movies['movieId'] == movie]['title'].values[0]
            print("Movie:", movie_title, "--> Predicted Rating:", rat)
        #compute all users satisfaction
        users_sud = compute_users_satisfaction(users, top_group)
        for i in range(len(users_sud)):
            users_satisfaction_iterations[i].append(users_sud[i])
        
        users_satisfaction_overall = compute_users_overall_satisfaction(users_satisfaction_iterations, iter)
        
        group_overall_satisfaction = sum(users_satisfaction_overall[user] for user in range(len(users)))/len(users)
        group_overall_disageement = (max(users_satisfaction_overall.values())) - (min(users_satisfaction_overall.values()))
        #print("\nGroup overall sadisfaction:", group_overall_satisfaction)
        print("Group overall disageement: ", group_overall_disageement)
        

users = [1, 3, 480]
iteration = 3
seq_aggregation(users, iteration)


Iteration:  0
Top 10 Movies:
Movie: Hush (2016) --> Predicted Rating: 4.971062119440762
Movie: Opera (1987) --> Predicted Rating: 4.91882532435635
Movie: The Cloverfield Paradox (2018) --> Predicted Rating: 4.887814582824376
Movie: Ivan's Childhood (a.k.a. My Name is Ivan) (Ivanovo detstvo) (1962) --> Predicted Rating: 4.882981421284085
Movie: Unbelievable Adventures of Italians in Russia (1974) --> Predicted Rating: 4.609132121155715
Movie: The Great Train Robbery (1903) --> Predicted Rating: 4.56979111350653
Movie: Dragon Ball Z: The History of Trunks (Doragon bôru Z: Zetsubô e no hankô!! Nokosareta chô senshi - Gohan to Torankusu) (1993) --> Predicted Rating: 4.457192915928266
Movie: Skin I Live In, The (La piel que habito) (2011) --> Predicted Rating: 4.457061709642226
Movie: Peeping Tom (1960) --> Predicted Rating: 4.4350452060414165
Movie: To Be or Not to Be (1942) --> Predicted Rating: 4.431458151523233
Group overall disageement:  0.5706565327123191

Iteration:  1
Top 10 Movies

In [18]:
#variance of the satisfaction between the single users' satisfaction and the mean
def get_users_satisfaction_variance(users_satisfaction, users):
    
    mean_satisfaction = sum(users_satisfaction) / len(users)
    sum_absolute_differences = sum(abs(s - mean_satisfaction) for s in users_satisfaction)
    disagreement = sum_absolute_differences / len(users)

    return disagreement



#modified version that use the variance
def my_sequence_aggregation(users, iteration):
    
    top_group={} #to memorize the movies recommended in the current itration
    old_iterations_moives={} #to memorize the movies recommended in the iteeration before
    users_satisfaction_iterations = {user: [] for user in range(len(users))}
    #get the two method aggregation
    movies_avg = average_aggregation(users)
    movies_least_misery = least_misery_aggregation(users)
    for iter in range(iteration):
        print("\nIteration: ", iter)
        # the first iteration has the recommendation of the average aggregation
        if iter == 0:
            top_group = movies_avg
        else:
            #compute previous iteration sadisfction alfa
            users_sadisfaction = compute_users_satisfaction(users,  top_group)
            alfa = get_users_satisfaction_variance(users_sadisfaction, users)
            new_recommendation = {}
            for movie, avg_rating in movies_avg.items():
                if movie not in old_iterations_moives: #to consider different movies for the new iteration
                    if movie in movies_least_misery:
                        min_rating = movies_least_misery[movie]
                        new_recommendation[movie] = avg_rating * (1 - alfa) + min_rating * alfa
            top_group = dict(sorted(new_recommendation.items(), key=lambda item: item[1], reverse=True))

        old_iterations_moives.update(dict(list(top_group.items())[:10])) #to memorize the movies predicted in the old iteration
        #top movies
        print("Top 10 Movies:")
        for movie, rat in dict(list(top_group.items())[:10]).items():
            movie_title = movies[movies['movieId'] == movie]['title'].values[0]
            print("Movie:", movie_title, "--> Predicted Rating:", rat)
        
        users_sut = compute_users_satisfaction(users,  dict(list(top_group.items())))
        for i in range(len(users_sut)):
            users_satisfaction_iterations[i].append(users_sut[i])
        
        users_satisfaction_overall = compute_users_overall_satisfaction(users_satisfaction_iterations, iter)

        group_overall_satisfaction = sum(users_satisfaction_overall[user] for user in range(len(users)))/len(users)
        group_overall_disageement = (max(users_satisfaction_overall.values())) - (min(users_satisfaction_overall.values()))
        #print("\nGroup overall satisfaction:", group_overall_satisfaction)
        print("Group overall disageement: ", group_overall_disageement)
        

users = [1, 3, 480]
iteration = 3
my_sequence_aggregation(users, iteration)


Iteration:  0
Top 10 Movies:
Movie: Hush (2016) --> Predicted Rating: 4.971062119440762
Movie: Opera (1987) --> Predicted Rating: 4.91882532435635
Movie: The Cloverfield Paradox (2018) --> Predicted Rating: 4.887814582824376
Movie: Ivan's Childhood (a.k.a. My Name is Ivan) (Ivanovo detstvo) (1962) --> Predicted Rating: 4.882981421284085
Movie: Unbelievable Adventures of Italians in Russia (1974) --> Predicted Rating: 4.609132121155715
Movie: The Great Train Robbery (1903) --> Predicted Rating: 4.56979111350653
Movie: Dragon Ball Z: The History of Trunks (Doragon bôru Z: Zetsubô e no hankô!! Nokosareta chô senshi - Gohan to Torankusu) (1993) --> Predicted Rating: 4.457192915928266
Movie: Skin I Live In, The (La piel que habito) (2011) --> Predicted Rating: 4.457061709642226
Movie: Peeping Tom (1960) --> Predicted Rating: 4.4350452060414165
Movie: To Be or Not to Be (1942) --> Predicted Rating: 4.431458151523233
Group overall disageement:  0.5706565327123191

Iteration:  1
Top 10 Movies

In [19]:
#variance of the satisfaction between the single users' satisfaction and the mean
def get_users_satisfaction_variance_norm(users_satisfaction, users):
    mean_satisfaction = sum(users_satisfaction) / len(users)
    sum_absolute_differences = sum(abs(s - mean_satisfaction) for s in users_satisfaction)
    disagreement = sum_absolute_differences / len(users)
    
    # Find the minimum and maximum possible disagreement values
    min_disagreement = 0  # Minimum possible disagreement
    max_disagreement = max(abs(s - mean_satisfaction) for s in users_satisfaction)  # Maximum possible disagreement

    # Normalize the disagreement using min-max normalization
    normalized_disagreement = (disagreement - min_disagreement) / (max_disagreement - min_disagreement)

    return normalized_disagreement


#another version with the weighted aggregation and the least misery
def my_seq_aggregation(users, iteration):
    
    top_group={}
    old_iterations_moives={}
    users_satisfaction_iterations = {user: [] for user in range(len(users))}
    #get the two method aggregation
    movies_weight = weighted_aggregation(users)
    movies_least_misery = least_misery_aggregation(users)
    for iter in range(iteration):
        print("\nIteration: ", iter)
        if iter == 0: # the first iteration use only the weighted method and memorize the rating such that it's bigger if the difference is smaller
            top_group = movies_least_misery
        else:
            new_recommendation = {}
            #compute previous iteration sadisfction alfa
            users_sadisfaction = compute_users_satisfaction(users,  top_group)
            alfa = get_users_satisfaction_variance_norm(users_sadisfaction, users)
            for movie, (avg_rating,diff) in movies_weight.items():
                if movie not in old_iterations_moives: #to consider different movies for the new iteration
                    if movie in movies_least_misery:
                        min_rating = movies_least_misery[movie]
                        new_recommendation[movie] = (avg_rating/diff) * alfa + min_rating * (1 - alfa)
            top_group = dict(sorted(new_recommendation.items(), key=lambda item: item[1], reverse=True))

        old_iterations_moives.update(dict(list(top_group.items())[:10])) #to memorize the movies predicted in the old iteration
        #top movies
        print("Top 10 Movies:")
        for movie, rat in dict(list(top_group.items())[:10]).items():
            movie_title = movies[movies['movieId'] == movie]['title'].values[0]
            print("Movie:", movie_title, "--> Predicted Rating:", rat)
        
        users_sat = compute_users_satisfaction(users,  dict(list(top_group.items())))
        for i in range(len(users_sat)):
            users_satisfaction_iterations[i].append(users_sat[i])
        
        users_satisfaction_overall = compute_users_overall_satisfaction(users_satisfaction_iterations, iter)
        
        group_overall_satisfaction = sum(users_satisfaction_overall[user] for user in range(len(users)))/len(users)
        group_overall_disageement = (max(users_satisfaction_overall.values())) - (min(users_satisfaction_overall.values()))
        #print("\nGroup overall satisfaction:", group_overall_satisfaction)
        print("Group overall disageement: ", group_overall_disageement)

users = [1, 3, 480]
iteration = 3
my_seq_aggregation(users, iteration)


Iteration:  0
Top 10 Movies:
Movie: Opera (1987) --> Predicted Rating: 4.435897435897436
Movie: Hush (2016) --> Predicted Rating: 4.269230769230769
Movie: The Cloverfield Paradox (2018) --> Predicted Rating: 4.185897435897436
Movie: Peeping Tom (1960) --> Predicted Rating: 4.002858871881985
Movie: Hachiko: A Dog's Story (a.k.a. Hachi: A Dog's Tale) (2009) --> Predicted Rating: 3.935897435897436
Movie: Skin I Live In, The (La piel que habito) (2011) --> Predicted Rating: 3.9358974358974357
Movie: Witchfinder General (Conquerer Worm, The) (1968) --> Predicted Rating: 3.9358974358974357
Movie: The Great Train Robbery (1903) --> Predicted Rating: 3.9206524589793657
Movie: Witches of Eastwick, The (1987) --> Predicted Rating: 3.7841442921880972
Movie: Maze Runner: Scorch Trials (2015) --> Predicted Rating: 3.7066252076548514
Group overall disageement:  0.41757695102908

Iteration:  1
Top 10 Movies:
Movie: Joy Ride (2001) --> Predicted Rating: 11.92338409990847
Movie: I Sell the Dead (2008)